In [1]:
import os

### Imports for pyOC

In [2]:
import pyOC
from pyOC import opencalphad as oc
from pyOC import PhaseStatus as phStat
from pyOC import GridMinimizerStatus as gmStat

### Setting verbosity (True or False - default), if set to yes, in particular, when getters are called the returned values are displayed in a comprehensive way

In [3]:
oc.setVerbosity(True)

### reading database (.tdb file)

In [4]:
tdbFile=os.environ.get('OCPUBLICDATA')+'/steel7.TDB'
oc.readtdb(tdbFile)

(OpenCalphad being verbose): reading /home/rl211391/develop/OpenCalphad/examples/macros//steel7.TDB
(OpenCalphad being verbose): component (6) names: ['C' 'CR' 'FE' 'MO' 'SI' 'V']


### Play with phase status

In [5]:
oc.setPhasesStatus(('* ',),phStat.Suspended)
phaseNames=('FCC_A1','M23C6','M6C')
oc.setPhasesStatus(phaseNames,phStat.Entered)

(OpenCalphad being verbose): modifying phases *  to status PhaseStatus.Suspended
(OpenCalphad being verbose): modifying phases FCC_A1 M23C6 M6C to status PhaseStatus.Entered


### Set pressure and temperature

In [6]:
oc.setPressure(1E5)
oc.setTemperature(1173)

(OpenCalphad being verbose): setting pressure to 1.00e+05 Pa
(OpenCalphad being verbose): setting temperature to 1173.00 K


### Set element molar amounts

In [7]:
elementMolarAmounts = {
	'C' : 0.04,
	'CR' : 0.06,
	'MO': 0.05,
	'SI': 0.003,
	'V': 0.01,
	'FE': 1.0-0.04-0.06-0.05-0.003-0.01
}
oc.setElementMolarAmounts(elementMolarAmounts)

(OpenCalphad being verbose): setting molar amount 0.0400 for element C (0)
(OpenCalphad being verbose): setting molar amount 0.0600 for element CR (1)
(OpenCalphad being verbose): setting molar amount 0.0500 for element MO (3)
(OpenCalphad being verbose): setting molar amount 0.0030 for element SI (4)
(OpenCalphad being verbose): setting molar amount 0.0100 for element V (5)
(OpenCalphad being verbose): setting molar amount 0.8370 for element FE (2)


### Calculate equilibrium without the grid-minimizer (equilibrium record is 'eq2')

In [8]:
oc.changeEquilibriumRecord('eq2')
oc.calculateEquilibrium(gmStat.Off)

(OpenCalphad being verbose): creating and selecting new equilibrium record 'eq2' ('EQ2')
(OpenCalphad being verbose): calculating equilibrium with grid minimizer GridMinimizerStatus.Off


### Calculate equilibrium with the grid-minimizer (equilibrium record is 'default equilibrium')

In [9]:
oc.changeEquilibriumRecord()
oc.calculateEquilibrium()

(OpenCalphad being verbose): selecting equilibrium record 'default equilibrium' ('DEFAULT_EQUILIBRIUM')
(OpenCalphad being verbose): calculating equilibrium with grid minimizer GridMinimizerStatus.On


### Retrieving Gibbs energies and comparing them

In [10]:
oc.changeEquilibriumRecord()
G=oc.getGibbsEnergy() # a scalar
oc.changeEquilibriumRecord('eq2')
G2=oc.getGibbsEnergy() # a scalar
print('G={0:e} (vs. without grid-minimizer: {1:e})'.format(G,G2))
oc.changeEquilibriumRecord()

(OpenCalphad being verbose): selecting equilibrium record 'default equilibrium' ('DEFAULT_EQUILIBRIUM')
(OpenCalphad being verbose): retrieving G: -5.767378e+04
(OpenCalphad being verbose): selecting equilibrium record 'eq2' ('EQ2')
(OpenCalphad being verbose): retrieving G: -5.760589e+04
(OpenCalphad being verbose): selecting equilibrium record 'default equilibrium' ('DEFAULT_EQUILIBRIUM')


G=-5.767378e+04 (vs. without grid-minimizer: -5.760589e+04)


### Retrieving chemical potentials

In [11]:
mu=oc.getChemicalPotentials() # a dictionary (keys are element names, values are chemical potentials)
print('mu_FE= ',mu['FE'])

(OpenCalphad being verbose): retrieving MU:
{
    "C": -40277.342035801805,
    "CR": -68893.39293504054,
    "FE": -55314.718539072106,
    "MO": -74567.11408055216,
    "SI": -196024.33490496737,
    "V": -131423.24634989392
}


mu_FE=  -55314.718539072106


### Retrieving equilibrium phases composition

In [12]:
phasesAtEquilibrium=oc.getPhasesAtEquilibrium() # a container class defined in pyOC.py
phaseElementComposition=phasesAtEquilibrium.getPhaseElementComposition() # a dictionary (keys are phase names) of dictionaries (keys are the element names, values are molar fractions)
print('n_C in FCC_A1#1 = ',phaseElementComposition['FCC_A1#1']['C'])
phaseSites=phasesAtEquilibrium.getPhaseSites() # a dictionary (keys are phase names, values are arrays of number of sites whose sizes depend on the number of sublattices)
print('a_i in FCC_A1#1 = ',phaseSites['FCC_A1#1'])
phaseConstituentComposition=phasesAtEquilibrium.getPhaseConstituentComposition() # a dictionary (keys are phase names) of dictionaries per sublattice (keys are the constituent names, values are molar fractions)
print('y_V^0 in liquid = ',phaseConstituentComposition['FCC_A1#1']['sublattice 0']['V'])

(OpenCalphad being verbose): phases at equilibrium:
phase molar amounts:
{
    "FCC_A1#1": 0.01920542894617253,
    "M23C6": 0.02977621280681491,
    "M6C": 0.09441638360236765,
    "FCC_A1_AUTO#2": 0.856601974644645
}
phase element composition:
{
    "FCC_A1#1": {
        "C": 0.45665297099480046,
        "CR": 0.03426417107693683,
        "FE": 0.0020841630748334116,
        "MO": 0.1443298386826512,
        "SI": 6.681137603878831e-10,
        "V": 0.3626688555026644
    },
    "M23C6": {
        "C": 0.2068965517241379,
        "CR": 0.3414624222993837,
        "FE": 0.3838209757158874,
        "MO": 0.06769132200309422,
        "SI": 0.0,
        "V": 0.00012872825749663693
    },
    "M6C": {
        "C": 0.14285714285714285,
        "CR": 0.06508399462251023,
        "FE": 0.38133540345297173,
        "MO": 0.39191933121811307,
        "SI": 0.0,
        "V": 0.018804127849262088
    },
    "FCC_A1_AUTO#2": {
        "C": 0.013519853563065565,
        "CR": 0.050232768562391776,

n_C in FCC_A1#1 =  0.45665297099480046
a_i in FCC_A1#1 =  [1.0, 1.0]
y_V^0 in liquid =  0.6674718663073684


### Retrieving constituent composition (this info is 'updated' each time the getPhasesAtEquilibrium method is called by adding constituents that were not present before)

In [13]:
constituentsDescription = oc.getConstituentsDescription() # a dictionary (keys are the constituent names) of dictionaries (keys are 'mass', 'charge' and 'elements', values are respectively the constituent molar mass, the constituent eletrical charge and a dictionary - keys are element names, values are stoichiometric coefficients)
print('m_V = ',constituentsDescription['V']['mass'])
print('q_V = ',constituentsDescription['V']['charge'])
print('stoi^V_V = ',constituentsDescription['V']['elements']['V'])

(OpenCalphad being verbose): constituents description:
{
    "CR": {
        "mass": 51.996,
        "charge": 0.0,
        "elements": {
            "CR": 1.0
        }
    },
    "FE": {
        "mass": 55.846999999999994,
        "charge": 0.0,
        "elements": {
            "FE": 1.0
        }
    },
    "MO": {
        "mass": 95.94,
        "charge": 0.0,
        "elements": {
            "MO": 1.0
        }
    },
    "SI": {
        "mass": 28.085,
        "charge": 0.0,
        "elements": {
            "SI": 1.0
        }
    },
    "V": {
        "mass": 50.941,
        "charge": 0.0,
        "elements": {
            "V": 1.0
        }
    },
    "C": {
        "mass": 12.011000000000001,
        "charge": 0.0,
        "elements": {
            "C": 1.0
        }
    },
    "VA": {
        "mass": 0.0,
        "charge": 0.0,
        "elements": {}
    }
}


m_V =  50.941
q_V =  0.0
stoi^V_V =  1.0
